# Setup

In [ ]:
# Run this locally. Binder dont need run.
!conda env create -n ict3104 --file environment.yml

In [ ]:
# Run this locally. Binder dont need run.
! conda activate ict3104 && .\venv\Scripts\Activate && pip install -r requirements.txt && pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html

# Example Use

In [ ]:
!cd ./pipeline && python test.py

# Import libraries

In [ ]:
# Upload File using ipyfilechooser library
from ipyfilechooser import FileChooser
# Get the root directory of the project
from pyprojroot import here
# Copy File
import shutil
# Widget Packages
import ipywidgets as widgets
# In case widget extension not working
# jupyter nbextension enable --py widgetsnbextension
import os

# Apply Module

# Video Upload using ipyfilechooser

In [ ]:
# from ipyfilechooser import FileChooser
starting_directory = './data'
chooser = FileChooser(starting_directory)
display(chooser)

# Upload Function to upload file to the data folder

In [ ]:
# Upload Function
# from pyprojroot import here
# import shutil
print(chooser.selected)
source = chooser.selected
# Source path
# Destination path
destination = (here("./data"))
 
# Copy file from the selected path
try:
    shutil.copy(source, destination)
    print("File copied successfully.")
 
# If source and destination are same
except shutil.SameFileError:
    print("Source and destination represents the same file.")
 
# If destination is a directory.
except IsADirectoryError:
    print("Destination is a directory.")
 
# If there is any permission issue
except PermissionError:
    print("Permission denied.")
 
# For other errors
except:
    print("Error occurred while copying file.")

In [ ]:
print(chooser.selected)

# Video Display

In [ ]:
from IPython.display import Video
full_path = chooser.selected
Video(full_path, embed=True, width=540)

In [ ]:
import os
from ipywidgets import interact, Dropdown

videoList = [] 

for x in os.listdir("./data/untrim/RGB"): 
    videoList += [x]
# print(videoList)

videos = Dropdown(options = videoList) 

# using a list as a means to extract variable defined in python function.
selected_video = []
 
@interact(video = videos) 
def print_city(video): 
    print(video)
    # list shall only contain one element, that is the current selected video.
    selected_video.clear()
    selected_video.append(str(video))


In [ ]:
from IPython.display import Video
full_path = chooser.selected
Video(full_path, embed=True, width=540)

# Selection

In [ ]:
"""
Select the pipeline in dropdown menu
"""
pipelines = ['ToyotaHome', 'Nvidia']

# Widgets
confirmButton = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)
pipelinesMenu = widgets.Dropdown(
    options=pipelines,
    value=pipelines[0],
    description='Select your pipeline:')
# Function
def selectWidgetSet(b):
    print("Selected: " , pipelinesMenu.value)

confirmButton.on_click(selectWidgetSet)
pipelineBox = widgets.VBox([widgets.HBox([pipelinesMenu, confirmButton])])
pipelineBox

In [ ]:
print(pipelinesMenu)